### In Case of Using CoLab : 구글드라이브 연계

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


★ setting point: Set the folder path where your data is stored.

example: "/content/gdrive/MyDrive/Colab Notebooks/llama/01. data preparation/dataset"


In [2]:
dataPath = "/content/gdrive/MyDrive/LLMClass/datasets/"
#dataPath = "./dataset/"
!ls /content/gdrive/MyDrive/LLMClass/datasets/

btsdata_kor.csv   llama2_gen_mycustom_data.ipynb  politec.json	   politec_pre.json
btsdata_kor.json  politec.csv			  politec_pre.csv  politec_pre.jsonl


### Install Library

In [3]:
# datasets: loads and manages data from hugging-face's community.
# jsonlines: handles json format data
!pip install datasets==2.16.1 jsonlines==4.0.0

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2023.10.0 which is incompatible.


### Login to access huggign face

In [4]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

### Colab 에서 huggingface hub 로그인이 잘 안되면 아래와 같이 cli login 으로 진행

In [21]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

In [ ]:
# Define installed libraries

In [8]:
import pandas as pd
import json
import jsonlines
from datasets import Dataset
import os

# ★ Hugging face 없이 csv 바로 튜닝데이터로 변환 ★ HF 호환

In [15]:
#csvfilepath = dataPath+"btsdata_kor.csv"
#jsonfilepath = dataPath+"btsdata_kor.json"
csvfilepath = dataPath+"politec.csv"
jsonfilepath = dataPath+"politec.json"

In [16]:
import pandas as pd

In [17]:
#csvdata = pd.read_csv(csvfilepath, encoding="ms949")
try:
    csvdata = pd.read_csv(csvfilepath, encoding="ms949")
except Exception as e:
    csvdata = pd.read_csv(csvfilepath, encoding="utf-8")

data_json = csvdata.to_json(jsonfilepath)

In [18]:
df = pd.read_json(jsonfilepath)

In [19]:
df.head(5)

,instruction,output,Unnamed: 2
0,폴리텍대학 광주캠퍼스 AI융합과 졸업 및 수료 후 주요 취업처는 어디인가요?,"1. 데이터 전처리, 머신러닝, 딥러닝 분야. 2. 인공지능 플랫폼 개발 및 구축 ...",NaN
1,폴리텍대학 광주캠퍼스 AI융합과의 학과장은 누구인가요?,김종민 교수님,NaN
2,폴리텍대학 광주캠퍼스 취득할 수 있는 자격증은 무엇이 있나요?,"정보처리기사, 빅데이터분석기사, 정보처리산업기사, 정보보안산업기사",NaN
3,폴리텍대학 광주캠퍼스 AI융합과의 교육목표는 무엇인가요?,소프트웨어와 AI알고리즘 융합 기술로 AI를 활용한 다양한 분야의 전문가 양성,NaN
4,폴리텍대학 광주캠퍼스 AI융합과는 어디에 있나요?,한국 폴리텍대학교 광주캠퍼스 제2공학관 3층,NaN


In [20]:
from datasets import Dataset

def format_example(row):
    return {
        'text': f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>

        {row['instruction']}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>

        {row['output']}<|eot_id|>"""
    }

# 판다스 데이터프레임을 데이터셋으로 변환하고, 포맷팅 함수 적용
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_example)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
dataset[2]

{'instruction': '폴리텍대학 광주캠퍼스 취득할 수 있는 자격증은 무엇이 있나요?',
 'output': '정보처리기사, 빅데이터분석기사, 정보처리산업기사, 정보보안산업기사',
 'Unnamed: 2': None,
 '__index_level_0__': 2,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n \n        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n \n        폴리텍대학 광주캠퍼스 취득할 수 있는 자격증은 무엇이 있나요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n \n        정보처리기사, 빅데이터분석기사, 정보처리산업기사, 정보보안산업기사<|eot_id|>'}

In [ ]:
# 필요시 허깅페이스에 업로드
#dataset.push_to_hub("rssaem/btsdata_kor")
dataset.push_to_hub("rssaem/llm_ko_politec")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rssaem/llm_ko_politec/commit/aa47bfb43695005601d50592300e71e5f82d4da6', commit_message='Upload dataset', commit_description='', oid='aa47bfb43695005601d50592300e71e5f82d4da6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rssaem/llm_ko_politec', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rssaem/llm_ko_politec'), pr_revision=None, pr_num=None)